In [2]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16351 sha256=f2fcb4951d82c7449ddcf9d02f3f8bd53bb9ec952c628687b7c9aaaaab278182
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [4]:
# Install packages
!pip install nltk
!pip install numpy
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
!pip install pandas

In [9]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-oh1scghg
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-oh1scghg
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456767 sha256=af55d7d2dd450e099a7029e6ecf8cf13289b878770e7cbc434689e4fe9952aac
  Stored in directory: /tmp/pip-ephem-wheel-cache-42i_ll5f/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT


In [5]:
import evaluate
import numpy as np
import rouge_score
import sacrebleu
from sacrebleu import CHRF
import ast

In [10]:
# Metrics
rouge_score = evaluate.load("rouge")
bleu_score = evaluate.load("bleu")
sacrebleu_score = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")
bleurt = evaluate.load("bleurt", module_type="metric")

In [11]:
from googletrans import Translator, LANGUAGES
import pandas as pd

def translate_text(input_text, target_language_code):
    try:
        translator = Translator()
        translation = translator.translate(input_text, dest=target_language_code)
        return translation.text
    except Exception as e:
        return str(e)

input_text = "Hello, how are you?"
target_language_code = "fr"

translated_text = translate_text(input_text, target_language_code)
print(f"Original Text: {input_text}")
print(f"Translated Text ({LANGUAGES[target_language_code]}): {translated_text}")

Original Text: Hello, how are you?
Translated Text (french): Bonjour comment allez-vous?


In [12]:
langs = {
    "english": "en",
    "russian": "ru",
    "hausa": "ha",
    "igbo": "ig",
    "swahili": "sw",
    "yoruba": "yo",
    "french": "fr",
    "arabic": "ar",
    "portuguese": "pt"
}

langs_list = ["ha", "ig", "sw", "yo", "fr", "ar", "pt"]

In [7]:
translated_text = translate_text("Hello, how are you?", langs['russian'])
print(f"Original Text: {input_text}")
print(f"Translated Text: {translated_text}")

Original Text: Hello, how are you?
Translated Text: Привет, как дела?


In [13]:
def evaluate(preds, refs):
  # score = chrf.compute(
  #   predictions=preds,
  #   references=refs,
  #   lowercase=True
  # )
  # print("CHRF: " + str(score))
  # print()

  # scores = rouge_score.compute(
  #     predictions=preds,
  #     references=refs,
  #     rouge_types=["rouge1", "rouge2", "rougeL"]
  # )

  # print("ROUGE:")
  # print(scores)
  # print()

  # score = bleu_score.compute(
  #     predictions=preds,
  #     references=refs
  # )
  # print("BLEU: " + str(score))
  # print()

  score = bleurt.compute(
      predictions=preds,
      references=refs
  )
  print("BLEURT: " + str(score))
  print()

In [14]:
def process_dataframe(df, target_language):
    translated_sentences = []
    original_sentences = []

    for index, row in df.iterrows():
        example_id = row['example_id']
        table_text = row['table_text']

        if '-en-' in example_id:
            try:
              parts = example_id.split("-")
              prefix = parts[0]
              suffix = parts[2]

              for i in range(len(table_text)):
                # Translate the English text to the target language
                translated_text = translate_text(table_text[i], target_language)
                original_sentence = df[df['example_id'] == prefix + '-' + target_language + '-' + suffix].values[0][6][i]

                translated_sentences.append(translated_text)
                original_sentences.append(original_sentence)

                # print(original_sentence)
                # print(translated_text)
                # print()
            except:
                pass

    evaluate(translated_sentences, original_sentences)

In [15]:
df = pd.read_json('train_skip_no_refs.json')

In [ ]:
for lang in langs_list:
  process_dataframe(df, lang)

BLEURT: {'scores': [-0.35313454270362854, -0.05484822764992714, -0.3033282160758972, -0.20150236785411835, -0.11567236483097076, -0.6993556022644043, -0.5160588622093201, -0.06984567642211914, 0.03107188269495964, -0.17793932557106018, -0.1408906877040863, -0.1408906877040863, -0.03954751417040825, -0.2561994194984436, -0.08612120151519775, 0.22934478521347046, 0.09946483373641968, -0.0016164816915988922, 0.009739916771650314, 0.1930178999900818, 0.15082889795303345, 0.041920412331819534, -0.3229343295097351, -0.06774768233299255, -0.18948408961296082, 0.020882543176412582, 0.1429494321346283, 0.09890320897102356, -0.08307500183582306, -0.19171053171157837, -0.21489761769771576, -0.13393482565879822, 0.02232196554541588, 0.1018889993429184, 0.2997893989086151, -0.26786860823631287, 0.07330627739429474, 0.17396700382232666, 0.14253973960876465, -0.21506990492343903, -0.28280138969421387, -0.20813491940498352, -0.32117417454719543, -0.5225469470024109, -0.6241950392723083, -0.41249531507